<center>
<h2> Импорт необходимых библиотек </h2>
</center>

In [ ]:
import numpy as np
import pandas as pd
import re

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

<center>
<h2> Подгрузим наши данные с репозитория проекта </h2>
</center>

In [ ]:
df_1 = pd.read_csv('https://raw.githubusercontent.com/ExuberantSebastian/AnDan_Python_Project/main/parser/DataFrames/GraphicsCards.csv')
df_2 = pd.read_csv('https://raw.githubusercontent.com/ExuberantSebastian/AnDan_Python_Project/main/parser/DataFrames/GraphicsCards_1.csv')

In [ ]:
Frames = [df_1, df_2]
df = pd.concat(Frames, ignore_index=True)
df.head()

,Видеокарта,Цена,комплектация,область применения,особенности,объем видеопамяти,тип памяти,частота памяти,шина обмена с памятью,частота видеопроцессора,разъемы и интерфейсы,тип подключения,поддержка технологий,разъем дополнительного питания,максимальное разрешение,количество поддерживаемых мониторов,техпроцесс,TDP,SLI/NVLink/CrossFire
0,Видеокарта SAPPHIRE RX580 8 Гб V2,9899,Retail,игровая,нет,8192 МБ,GDDR5,1284 МГц,256 бит,1340 МГц,"выход DVI, выход DisplayPort, выход HDMI",PCI Express 3.0,нет,8 pin,7680x4320,не поддерживается,NaN,NaN,NaN
1,Видеокарта ASUS Dual GeForce RTX 3060 V2 OC Ed...,36500,Retail,игровая,необходимость дополнительного питания,12288 МБ,GDDR6,15000 МГц,192 бит,1837 МГц,"выход DisplayPort, выход DisplayPort x 3, выхо...",PCI Express 4.0,"поддержка CUDA, поддержка HDCP, поддержка Vulk...",8 pin,7680x4320,4,8 нм,170 Вт,NaN
2,Видеокарта GIGABYTE GeForce RTX 3070 Ti GAMING...,62000,Retail,игровая,NaN,8192 МБ,GDDR6X,19000 МГц,256 бит,1830 МГц,"выход DisplayPort x 2, выход HDMI x 2, интерфе...",PCI Express 4.0,NaN,8 pin + 8 pin,7680x4320,4,8 нм,290 Вт,NaN
3,Видеокарта БУ AMD RADEON RX 580 8Gb OC ASRock ...,8700,OEM,игровая,необходимость дополнительного питания,8192 МБ,GDDR5,8000 МГц,256 бит,1424 МГц,"выход DVI, выход DisplayPort x 3, выход HDMI",PCI Express 3.0,NaN,6 pin,7680x4320,5,14 нм,NaN,NaN
4,"Видеокарта MSI GeForce RTX 3050 GAMING X 8G, R...",30734,Retail,игровая,необходимость дополнительного питания,8192 МБ,GDDR6,14000 МГц,128 бит,1550 МГц,"выход DisplayPort x 3, выход HDMI",PCI Express 4.0,"поддержка CUDA, поддержка HDCP, поддержка трас...",8 pin,7680x4320,4,8 нм,130 Вт,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135 entries, 0 to 1134
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Видеокарта                           1135 non-null   object
 1   Цена                                 1135 non-null   object
 2   комплектация                         704 non-null    object
 3   область применения                   743 non-null    object
 4   особенности                          361 non-null    object
 5   объем видеопамяти                    1054 non-null   object
 6   тип памяти                           1072 non-null   object
 7   частота памяти                       815 non-null    object
 8   шина обмена с памятью                987 non-null    object
 9   частота видеопроцессора              878 non-null    object
 10  разъемы и интерфейсы                 992 non-null    object
 11  тип подключения                      885 no

*Из представленной выше информации можно сделать вывод, что в наших данных немало пропусков, от которых придётся избавляться самыми различными способами*

Для удобства и деления задачи на сферы ответственности по команде, обработка датафрейма была разделена обработку числовых и нечисловых признаков

<center>

## Обработка нечисловых признаков ##

</center>

Мы постепенно выясним все болячки фрейма, избавимся от аномальных значений и приведём данные в адекватный вид

In [ ]:
df['тип памяти'].value_counts() # Видеокарты могут иметь только память типа GDDR или HBM

GDDR6          486
GDDR6X         260
GDDR5          196
GDDR3           82
DDR3            14
GDDR4           11
GDDR5X           8
GDDR2            5
HBM2             5
DDR3, GDDR3      3
DDR5             1
DDR4             1
Name: тип памяти, dtype: int64

In [ ]:
df['тип памяти'] = df['тип памяти'].str.replace('DDR3, GDDR3', 'GDDR3').str.replace('DDR3', 'GDDR3').str.replace('DDR5', 'GDDR5').str.replace('DDR4', 'GDDR4') # Косметические операции
df['тип памяти'] = df['тип памяти'].str.replace('GG', 'G') 


Теперь избавимся от выбросов

In [ ]:
df_1.loc[df_1['тип памяти'].isnull() == True][:15] # Судя по названиям это встройки, они нам для анализа не нужны

,Видеокарта,Цена,комплектация,область применения,особенности,объем видеопамяти,тип памяти,частота памяти,шина обмена с памятью,частота видеопроцессора,разъемы и интерфейсы,тип подключения,поддержка технологий,разъем дополнительного питания,максимальное разрешение,количество поддерживаемых мониторов,техпроцесс,TDP,SLI/NVLink/CrossFire
107,Видеокарта ASUS GeForce GTX 960 STRIX 2GB GDDR...,7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,KFA2 GeForce GTX 1660 Super EX White (1-Click OC),16600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,Видеокарта ZOTAC GeForce GTS 450 810Mhz PCI-E ...,1755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,Видеокарта Sapphire Radeon X1650 Pro 450Mhz PC...,585₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,Видеокарта 12Gb PCI-E GDDR6 GIGABYTE RADEON RX...,35000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,Видеокарта Palit GeForce GTS 250 702Mhz PCI-E ...,1404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,Графический ускоритель NVIDIA Tesla V100 с тен...,347700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,Видеокарта BFG GeForce 8800 GTS 675Mhz PCI-E 5...,1170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,Gainward GeForce GT 220 635Mhz PCI-E 2.0 512Mb...,1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,Видеокарта 128Mb AGP DDR D-Sub+DVI+TV Out ATI ...,650₽,NaN,NaN,NaN,128 МБ,NaN,NaN,NaN,NaN,выход DVI,AGP,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['тип памяти'].fillna('f', inplace=True)
df = df[df['тип памяти'] != 'f'] # так-то лучше

Данные значения были удалены, т.к. представляли из себя "встройки", которые даже при наличии у них необходимых характеристик исказили бы нашу модель

Далее мы обработаем оставшиеся нечисловые значения

In [ ]:
df['область применения'].value_counts() # подавляющее большинство - игровые (что в принципе отражает реальность)

игровая             720
профессиональная     21
Name: область применения, dtype: int64

In [ ]:
df['комплектация'].value_counts() # списки и предсталяют собой ретейл

Retail                                                                             648
OEM                                                                                 43
Bulk                                                                                 6
Видеокарта, инструкция, диск с ПО                                                    2
Видеокарта, диск с ПО, инструкция, кабель-адаптер (опция)                            2
Видеокарта, инструкция, диск с ПО, кабель-адаптер питания (опция)                    1
16-pin to 4x 8-pin пареходник, VGA держатель, ARGB SYNC кабель                       1
Видеокарта, диск с ПО (опция), инструкция, кабель-переходник питания, кронштейн      1
Name: комплектация, dtype: int64

*Пару слов касательно написанного ниже:*

Т.к. об осбенностях товара не было написано, мы считаем, что их и нет <br>
Большая часть видеокарт - игровые, но это не значит, что они не годятся для работы, просто особенности маркировки <br>
У большей части современных видеокарт, из которых в основном и состоит наш массив - подключение 4 версии (Современные видеокарты опознавались по аббревиатуре 'RTX') <br>
Количество поддерживаемых мониторов у современных видеокарт всегда начинается от 2, если об этом не было сказано, можно предполагать что разъёмов 2 или 3 <br>
Чаще всего у видеокарт присутствует 8 pin дополнительного питания <br>
Большая часть видеокарт, которая имеет пропуск на максимально допустимом разрешении может работать на 4096x2160

In [ ]:
df['особенности'].fillna('нет', inplace=True)

df['область применения'].fillna('игровая', inplace=True)

df['комплектация'].fillna('Retail', inplace=True)
df['комплектация'].loc[(df['комплектация'] != 'Retail') & (df['комплектация'] != 'OEM') & (df['комплектация'] != 'Bulk')] = 'Retail'

df['тип подключения'].loc[(df['тип подключения'].isnull() == True) & (df['Видеокарта'].str.contains('RTX')) == True] = 'PCI Express 4.0'
df['тип подключения'].fillna('PCI Express 3.0', inplace=True)

df['количество поддерживаемых мониторов'] = df['количество поддерживаемых мониторов'].str.replace('не поддерживается', '2')
df['количество поддерживаемых мониторов'] = df['количество поддерживаемых мониторов'].str.replace('3, 4', '4')
df['количество поддерживаемых мониторов'].fillna('2', inplace=True)

df['разъем дополнительного питания'] = df['разъем дополнительного питания'].str.replace('6pin', '6 pin')
df['разъем дополнительного питания'] = df['разъем дополнительного питания'].str.replace('[()]', '')
df['разъем дополнительного питания'] = df['разъем дополнительного питания'].str.replace('0 нет pin', 'Нет')
df['разъем дополнительного питания'].fillna('8 pin', inplace=True)

df['поддержка технологий'].fillna('нет', inplace=True)

df['разъемы и интерфейсы'].fillna('выход DVI, выход DisplayPort, выход HDMI', inplace=True)

df['максимальное разрешение'].fillna('4096x2160', inplace=True)

In [ ]:
df['разъемы и интерфейсы'].value_counts()

выход DisplayPort x 3, выход HDMI                                                                    295
выход DVI, выход DisplayPort, выход HDMI                                                             211
выход DVI, выход HDMI, выход VGA                                                                      82
выход HDMI                                                                                            64
выход DisplayPort, выход HDMI                                                                         57
                                                                                                    ... 
выход DVI, выход HDMI, выход VGA, PCI-E, PCI-E                                                         1
выход DisplayPort x 2, выход DisplayPort x 3, выход HDMI x 2                                           1
выход DisplayPort, выход DisplayPort x 2, выход HDMI, выход HDMI x 2, интерфейс USB Type-C, PCI-E      1
выход DisplayPort, выход HDMI x 3                      

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1072 entries, 0 to 1134
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Видеокарта                           1072 non-null   object
 1   Цена                                 1072 non-null   object
 2   комплектация                         1072 non-null   object
 3   область применения                   1072 non-null   object
 4   особенности                          1072 non-null   object
 5   объем видеопамяти                    1025 non-null   object
 6   тип памяти                           1072 non-null   object
 7   частота памяти                       813 non-null    object
 8   шина обмена с памятью                982 non-null    object
 9   частота видеопроцессора              877 non-null    object
 10  разъемы и интерфейсы                 1072 non-null   object
 11  тип подключения                      1072 n

<center>

## Обработка числовых признаков ##

</center>

Числовые характеристики являются чуть ли не определяющими для цены той или иной видеокарты, ниже мы переведём их в числовой формат и заполним пропущенные значения, необходимые для дальнейшего анализа, отдельного комментария пожалуй заслуживает только шина, у которой присутствует два аномальных значения

In [ ]:
df['Цена'] = df['Цена'].str.replace('₽',' ') # Реплейс проходится по всем значениям, которые мешают обработке
df['Цена'] = pd.to_numeric(df['Цена'])
price = df['Цена']
price = pd.DataFrame(price)

df['объем видеопамяти'] = df['объем видеопамяти'].str.replace('МБ',' ')
df['объем видеопамяти'] = pd.to_numeric(df['объем видеопамяти'])
video_memory = df['объем видеопамяти']

df['частота памяти'] = df['частота памяти'].str.replace('МГц',' ')
df['частота памяти'] = pd.to_numeric(df['частота памяти'])
memory_frequency = df['частота памяти']

df['частота видеопроцессора'] = df['частота видеопроцессора'].str.replace('МГц',' ')
df['частота видеопроцессора'] = pd.to_numeric(df['частота видеопроцессора'])
videoprocessor_frequency = df['частота видеопроцессора']

df['техпроцесс'] = df['техпроцесс'].str.replace('нм',' ')
df['техпроцесс'] = pd.to_numeric(df['техпроцесс'])
techproces = df['техпроцесс']

df['шина обмена с памятью'].loc[df['шина обмена с памятью'] == '128 бит, 64 бит'] = '128 бит' # эта аномальная строка ломает код
df['шина обмена с памятью'] = df['шина обмена с памятью'].str.replace('бит',' ')
df['шина обмена с памятью'] = pd.to_numeric(df['шина обмена с памятью'])
tyre = df['шина обмена с памятью']

features_float = price.join(video_memory)
features_float = features_float.join(memory_frequency)
features_float = features_float.join(videoprocessor_frequency)
features_float = features_float.join(techproces)
features_float = features_float.join(tyre)

features_float

,Цена,объем видеопамяти,частота памяти,частота видеопроцессора,техпроцесс,шина обмена с памятью
0,9899,8192.0,1284.0,1340.0,NaN,256.0
1,36500,12288.0,15000.0,1837.0,8.0,192.0
2,62000,8192.0,19000.0,1830.0,8.0,256.0
3,8700,8192.0,8000.0,1424.0,14.0,256.0
4,30734,8192.0,14000.0,1550.0,8.0,128.0
...,...,...,...,...,...,...
1130,31190,8192.0,NaN,NaN,NaN,NaN
1131,90230,12288.0,2610.0,1920.0,5.0,192.0
1132,79830,12288.0,21000.0,1920.0,NaN,192.0
1133,65367,10240.0,NaN,NaN,NaN,NaN


In [ ]:
features_float.head(25)

,Цена,объем видеопамяти,частота памяти,частота видеопроцессора,техпроцесс,шина обмена с памятью
0,9899,8192.0,1284.0,1340.0,NaN,256.0
1,36500,12288.0,15000.0,1837.0,8.0,192.0
2,62000,8192.0,19000.0,1830.0,8.0,256.0
3,8700,8192.0,8000.0,1424.0,14.0,256.0
4,30734,8192.0,14000.0,1550.0,8.0,128.0
5,37500,12288.0,15000.0,1837.0,8.0,192.0
6,107854,16384.0,22400.0,2505.0,5.0,256.0
7,19700,8192.0,14000.0,1665.0,12.0,256.0
8,9800,8192.0,8000.0,1591.0,12.0,256.0
9,13390,8192.0,8000.0,1506.0,16.0,256.0


Частота памяти и видеопроцесса более хаотичные величины и их можно заменить на среднее значение, однако объём видеопамяти - это важнейшая характеристика, которая вероятно сильно влияет на цену, чаще всего она даже указывается в названии 

Далее мы заполним пропуски по категориям, заменяем медианами, т.к. в данном случае это будет наиболее логичное решение

In [ ]:
features_float.corr() # ЧТД

,Цена,объем видеопамяти,частота памяти,частота видеопроцессора,техпроцесс,шина обмена с памятью
Цена,1.000000,0.744677,0.330956,0.385411,-0.294511,0.712183
объем видеопамяти,0.744677,1.000000,0.547058,0.580745,-0.476953,0.456871
частота памяти,0.330956,0.547058,1.000000,0.760070,-0.659835,0.117892
частота видеопроцессора,0.385411,0.580745,0.760070,1.000000,-0.695812,0.120666
техпроцесс,-0.294511,-0.476953,-0.659835,-0.695812,1.000000,-0.040909
шина обмена с памятью,0.712183,0.456871,0.117892,0.120666,-0.040909,1.000000


Обратная корреляция у техпроцесса - это нормально, ибо чем он меньше - тем быстрее работает видеокарта

In [ ]:
# Цикл для заполнения пустоты 

Memory = ['10G', '12G', '16G', '24G', '20G', '2G', '4G', '6G', '8G', '3G', '1G']

for i in range(len(Memory)):
    a = re.findall(r'\d+', Memory[i]) 

    df['объем видеопамяти'].loc[(df['объем видеопамяти'].isnull() == True) & (df['Видеокарта'].str.contains(Memory[i]) == True)] = int(a[0])*1024.0

# Ещё пара значений с кривыми названиями

df['объем видеопамяти'].loc[(df['объем видеопамяти'].isnull() == True) & (df['Видеокарта'].str.contains('3050') == True)] = 8192.0
df['объем видеопамяти'].loc[(df['объем видеопамяти'].isnull() == True) & (df['Видеокарта'].str.contains('Palit') == True)] = 6*1024.0

# Остатки нам не нужны

df['объем видеопамяти'].fillna('f', inplace=True)
df = df[df['объем видеопамяти'] != 'f']

In [ ]:
df.loc[(df['объем видеопамяти'].isnull() == True)] # Так то лучше

,Видеокарта,Цена,комплектация,область применения,особенности,объем видеопамяти,тип памяти,частота памяти,шина обмена с памятью,частота видеопроцессора,разъемы и интерфейсы,тип подключения,поддержка технологий,разъем дополнительного питания,максимальное разрешение,количество поддерживаемых мониторов,техпроцесс,TDP,SLI/NVLink/CrossFire


In [ ]:
# Для более грамотного заполнения пробелов сгруппируем нашу выборку

rate_mem = df.groupby('объем видеопамяти')['частота памяти'].median()

# Оказалось, что у некоторых категорий вообще нет данных, придётся заполнять вручную

df['частота памяти'].loc[(df['частота памяти'].isnull() == True) & (df['объем видеопамяти'] == 3072.0)] = 4000.0
df['частота памяти'].loc[(df['частота памяти'].isnull() == True) & (df['объем видеопамяти'] == 5120.0)] = 8000.0
df['частота памяти'].loc[(df['частота памяти'].isnull() == True) & (df['объем видеопамяти'] == 20000.0)] = 20000.0
df['частота памяти'].loc[(df['частота памяти'].isnull() == True) & (df['объем видеопамяти'] == 81920.0)] = 14000.0

# Составим цикл, который заполнит оставшиеся пропуски

for i, j in rate_mem.items():
    df['частота памяти'].loc[(df['частота памяти'].isnull() == True) & (df['объем видеопамяти'] == i)] = j
    

In [ ]:
df.groupby('объем видеопамяти')['частота памяти'].median()

объем видеопамяти
256.0        800.0
512.0       1600.0
1024.0      1333.0
2000.0      3671.0
2048.0      4000.0
3072.0      4000.0
4096.0      8000.0
5120.0      8000.0
6000.0     14000.0
6144.0     14000.0
8000.0     14000.0
8192.0     14000.0
10000.0    19000.0
10240.0    19000.0
11264.0    11010.0
12000.0    21000.0
12288.0    16000.0
16000.0    22400.0
16384.0    18000.0
20000.0    20000.0
20480.0     2400.0
24000.0    20000.0
24567.0    21000.0
24576.0    21000.0
32768.0      878.0
49152.0    14000.0
81920.0    14000.0
Name: частота памяти, dtype: float64

In [ ]:
df.loc[df['частота памяти'].isnull() == True]

,Видеокарта,Цена,комплектация,область применения,особенности,объем видеопамяти,тип памяти,частота памяти,шина обмена с памятью,частота видеопроцессора,разъемы и интерфейсы,тип подключения,поддержка технологий,разъем дополнительного питания,максимальное разрешение,количество поддерживаемых мониторов,техпроцесс,TDP,SLI/NVLink/CrossFire


In [ ]:
df.groupby('объем видеопамяти')['частота видеопроцессора'].median() 

объем видеопамяти
256.0       400.0
512.0       650.0
1024.0      755.0
2000.0      945.0
2048.0      954.0
3072.0        NaN
4096.0     1410.0
5120.0        NaN
6000.0     1365.0
6144.0     1530.0
8000.0     1728.5
8192.0     1665.0
10000.0    1440.0
10240.0    1755.0
11264.0    1544.0
12000.0    2230.0
12288.0    1920.0
16000.0    2205.0
16384.0    2330.0
20000.0    2037.5
20480.0    2000.0
24000.0    2500.0
24567.0    2520.0
24576.0    2282.5
32768.0    1246.0
49152.0    1395.0
81920.0       NaN
Name: частота видеопроцессора, dtype: float64

In [ ]:
# Повторим процедуру

rate_proc = df.groupby('объем видеопамяти')['частота видеопроцессора'].median()

# Оказалось, что у некоторых категорий вообще нет данных, придётся заполнять вручную, где-то я это уже видел

df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == 3072.0)] = 954.0
df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == 5120.0)] = 1530.0
df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == 81920.0)] = 1395.0

# Составим цикл, который заполнит оставшиеся пропуски

for i, j in rate_proc.items():
    df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == i)] = j

In [ ]:
rate_proc = df.groupby('объем видеопамяти')['частота видеопроцессора'].median()

# Оказалось, что у некоторых категорий вообще нет данных, придётся заполнять вручную, где-то я это уже видел

df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == 3072.0)] = 954.0
df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == 5120.0)] = 1530.0
df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == 81920.0)] = 1395.0

# Составим цикл, который заполнит оставшиеся пропуски

for i, j in rate_proc.items():
    df['частота видеопроцессора'].loc[(df['частота видеопроцессора'].isnull() == True) & (df['объем видеопамяти'] == i)] = j

In [ ]:
df.groupby('объем видеопамяти')['техпроцесс'].median()

объем видеопамяти
256.0      90.0
512.0      47.5
1024.0     40.0
2000.0     28.0
2048.0     28.0
3072.0      NaN
4096.0     12.0
5120.0     16.0
6000.0     12.0
6144.0     12.0
8000.0      8.0
8192.0      8.0
10000.0     NaN
10240.0     8.0
11264.0    16.0
12000.0     7.0
12288.0     8.0
16000.0     4.0
16384.0     7.0
20000.0     5.0
20480.0     5.0
24000.0     5.0
24567.0     5.0
24576.0     5.0
32768.0    12.0
49152.0    12.0
81920.0     NaN
Name: техпроцесс, dtype: float64

In [ ]:
# Оказалось, что у некоторых категорий вообще нет данных, придётся заполнять вручную, где-то я это уже видел

df['техпроцесс'].loc[(df['техпроцесс'].isnull() == True) & (df['объем видеопамяти'] == 3072.0)] = 12.0
df['техпроцесс'].loc[(df['техпроцесс'].isnull() == True) & (df['объем видеопамяти'] == 10000.0)] = 8.0
df['техпроцесс'].loc[(df['техпроцесс'].isnull() == True) & (df['объем видеопамяти'] == 81920.0)] = 12.0

# Повторение - мать учения

proc = df.groupby('объем видеопамяти')['техпроцесс'].median()

# Составим цикл, который заполнит оставшиеся пропуски

for i, j in proc.items():
    df['техпроцесс'].loc[(df['техпроцесс'].isnull() == True) & (df['объем видеопамяти'] == i)] = j

In [ ]:
df.groupby('объем видеопамяти')['шина обмена с памятью'].median()

объем видеопамяти
256.0       128.0
512.0       128.0
1024.0       64.0
2000.0       64.0
2048.0       64.0
3072.0      192.0
4096.0      128.0
5120.0      160.0
6000.0      192.0
6144.0      192.0
8000.0      256.0
8192.0      256.0
10000.0     320.0
10240.0     320.0
11264.0     352.0
12000.0     192.0
12288.0     192.0
16000.0     256.0
16384.0     256.0
20000.0     320.0
20480.0     320.0
24000.0     384.0
24567.0     384.0
24576.0     384.0
32768.0    4096.0
49152.0     384.0
81920.0       NaN
Name: шина обмена с памятью, dtype: float64

In [ ]:
# Оказалось, что у некоторых категорий вообще нет данных, придётся заполнять вручную, где-то я это уже видел

df['шина обмена с памятью'].loc[(df['шина обмена с памятью'].isnull() == True) & (df['объем видеопамяти'] == 81920.0)] = 384.0

# Повторение повторения - бабушка учения

proc = df.groupby('объем видеопамяти')['шина обмена с памятью'].median()

# Составим цикл, который заполнит оставшиеся пропуски

for i, j in proc.items():
    df['шина обмена с памятью'].loc[(df['шина обмена с памятью'].isnull() == True) & (df['объем видеопамяти'] == i)] = j

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1064 entries, 0 to 1134
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Видеокарта                           1064 non-null   object 
 1   Цена                                 1064 non-null   int64  
 2   комплектация                         1064 non-null   object 
 3   область применения                   1064 non-null   object 
 4   особенности                          1064 non-null   object 
 5   объем видеопамяти                    1064 non-null   object 
 6   тип памяти                           1064 non-null   object 
 7   частота памяти                       1064 non-null   float64
 8   шина обмена с памятью                1064 non-null   float64
 9   частота видеопроцессора              1064 non-null   float64
 10  разъемы и интерфейсы                 1064 non-null   object 
 11  тип подключения               

Осталось только удались пару лишних столбцов и первичная обработка закончена

In [ ]:
df.drop('TDP', axis=1, inplace=True)
df.drop('SLI/NVLink/CrossFire', axis=1, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1064 entries, 0 to 1134
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Видеокарта                           1064 non-null   object 
 1   Цена                                 1064 non-null   int64  
 2   комплектация                         1064 non-null   object 
 3   область применения                   1064 non-null   object 
 4   особенности                          1064 non-null   object 
 5   объем видеопамяти                    1064 non-null   object 
 6   тип памяти                           1064 non-null   object 
 7   частота памяти                       1064 non-null   float64
 8   шина обмена с памятью                1064 non-null   float64
 9   частота видеопроцессора              1064 non-null   float64
 10  разъемы и интерфейсы                 1064 non-null   object 
 11  тип подключения               

Наконец-то наш датафрейм готов для дальнейшей обработки, но об этом в следующем файле

In [ ]:
df.to_csv('GraphicsCards.csv', index=False)